In [1]:
# lettura di tutti i documenti da una cartella

from langchain_community.document_loaders import DirectoryLoader  # pip install unstructured
                                                                  # https://pypi.org/project/python-magic-bin/0.4.14/#files scaricare e installare la wheel corretta

loader = DirectoryLoader('data/dialoghi', glob="**/*.txt")
documents = loader.load()

for doc in documents:
    print(doc, "\n\n")

page_content='Al negozio di abbigliamento Commesso - Buongiorno, posso aiutarla? Cliente - Sì, grazie. Ho visto un paio di pantaloni neri in vetrina, posso provarli? Commesso - Certo, che taglia? Cliente - Porto una 50 Commesso - Eccoli qua! Cliente - Dove sono i camerini? Commesso - I camerini sono là in fondo a destra, accanto alle scale Cliente - Perfetto, grazie

Commesso - Come vanno? Cliente - Il modello mi piace, ma sono un po' stretti. Posso provare una taglia più larga? Commesso - Oh, mi dispiace, abbiamo finito la taglia 52 in questo colore, vuole provare lo stesso modello in marrone? Cliente - No, grazie, il marrone proprio non mi piace. Non avete altri colori in questa taglia? Commesso - Allora, nella taglia 52 abbiamo il marrone, il rosso e il grigio. Cliente - Vabbè, li provo in grigio, vediamo come mi stanno.

Cliente - Ho provato i pantaloni, anche in grigio sono proprio belli e la taglia è perfetta! Quanto costano? Commesso - Costano 85€ Cliente - Ma non sono in sconto

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage._lc_store import create_kv_docstore  #  <---
from langchain.storage import LocalFileStore

# Text-Splitter per i chunk di documento più grandi, da usare come dati recuperati
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=350, chunk_overlap=150)

# Secondo Text-Splitter, per la creazione di chunk più piccoli su cui indicizzare le ricerche
child_splitter = RecursiveCharacterTextSplitter(chunk_size=85, chunk_overlap=15)

vectorstore = Chroma(collection_name="split_parents", embedding_function=OpenAIEmbeddings())

# dizionario su filesystem utilizzato per recuperare i documenti più grandi, una volta
# identificati tramite ricerca sui chunk più piccoli
fs = LocalFileStore("./storage_locale")
store = create_kv_docstore(fs)

In [4]:
# ParentDocumentRetriever gestisce automaticamente
# il "doppio livello" di granularità dei chunk

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

retriever.add_documents(documents)

In [5]:
# dai 5 documenti di partenza, vengono creati 
# 57 chunk da 350 caratteri massimo
# memorizzati in `store`, mentre i chunk più
# piccoli, da massimo 85 caratteri, sono
# memorizzati nel vector-store
len(list(store.yield_keys()))

228

In [6]:
store.store

In [7]:
store.store.root_path

WindowsPath('C:/Cloud/GitHub/LLMs_MasterClass/04 - RAG/storage_locale')

In [8]:
# accedendo al VectorStore (Chroma), tramite similarity search, la ricerca viene effettuata
# sui chunk estratti più piccoli
sub_docs = vectorstore.similarity_search("in quale dialogo ci sono delle mamme che parlano?")

sub_docs[0]

Document(metadata={'doc_id': '31fa77ae-3627-46f4-927c-a82b7051211a', 'source': 'data\\dialoghi\\dialogo3.txt'}, page_content='Mamma 2 - Qualcuna di voi sa qualcosa del prossimo incontro con i genitori? Mamma 3 -')

In [9]:
for s in sub_docs:
    print(s)

page_content='Mamma 2 - Qualcuna di voi sa qualcosa del prossimo incontro con i genitori? Mamma 3 -' metadata={'doc_id': '31fa77ae-3627-46f4-927c-a82b7051211a', 'source': 'data\\dialoghi\\dialogo3.txt'}
page_content='i nonni. È da un po' che non li vediamo e i bambini ne sentono la mancanza. Mamma 1' metadata={'doc_id': '99f4ae34-61da-4fe8-84bf-b5b529544150', 'source': 'data\\dialoghi\\dialogo3.txt'}
page_content='i nonni. È da un po' che non li vediamo e i bambini ne sentono la mancanza. Mamma 1 -' metadata={'doc_id': '8ffc14fc-a90a-402f-9561-483006567cff', 'source': 'data\\dialoghi\\dialogo3.txt'}
page_content='Mamma 2 - Cambiando argomento, cosa farete questo fine settimana? Mamma 3 - Noi' metadata={'doc_id': '99f4ae34-61da-4fe8-84bf-b5b529544150', 'source': 'data\\dialoghi\\dialogo3.txt'}


In [10]:
# il retriever ritorna invece il chunk di documento più grande
retrieved_docs = retriever.invoke("in quale dialogo ci sono delle mamme che parlano?")

retrieved_docs[0]

Document(metadata={'source': 'data\\dialoghi\\dialogo3.txt'}, page_content='Mamma 2 - Qualcuna di voi sa qualcosa del prossimo incontro con i genitori? Mamma 3 - Dovrebbe essere la prossima settimana, credo. Devo ancora controllare il calendario degli eventi. Mamma 4 - Sì, è giovedì prossimo alle 17:00. Hanno detto che parleranno dei progressi dei bambini e dei prossimi progetti scolastici. Mamma 1 - Perfetto, così posso')

In [11]:
for r in retrieved_docs:
    print(r)

page_content='Mamma 2 - Qualcuna di voi sa qualcosa del prossimo incontro con i genitori? Mamma 3 - Dovrebbe essere la prossima settimana, credo. Devo ancora controllare il calendario degli eventi. Mamma 4 - Sì, è giovedì prossimo alle 17:00. Hanno detto che parleranno dei progressi dei bambini e dei prossimi progetti scolastici. Mamma 1 - Perfetto, così posso' metadata={'source': 'data\\dialoghi\\dialogo3.txt'}
page_content='Mamma 2 - Cambiando argomento, cosa farete questo fine settimana? Mamma 3 - Noi andiamo al parco, sperando che il tempo sia bello. I bambini hanno bisogno di correre un po'. Mamma 4 - Noi invece visiteremo i nonni. È da un po' che non li vediamo e i bambini ne sentono la mancanza. Mamma 1 - Noi pensavamo di andare al cinema. C'è un nuovo film per' metadata={'source': 'data\\dialoghi\\dialogo3.txt'}
page_content='i nonni. È da un po' che non li vediamo e i bambini ne sentono la mancanza. Mamma 1 - Noi pensavamo di andare al cinema. C'è un nuovo film per bambini che